# Data discovery RAW

In [2]:
%load_ext raw_magic

In [3]:
%buckets_register raw-tutorial

API error: S3 credentials already exists


### Data discovery
First let's list a folder in S3 using the `ls` command

In [4]:
%%query
ll("s3://raw-tutorial/inference")

Let's look at the content of the csv file

In [4]:
%%query
SELECT * from read_lines("s3://raw-tutorial/inference/airports.csv") LIMIT 10

string
"""AirportID"",""Name"",""City"",""Country"",""IATA_FAA"",""ICAO"",""Latitude"",""Longitude"",""Altitude"",""Timezone"",""DST"",""TZ"""
"1,""Goroka"",""Goroka"",""Papua New Guinea"",""GKA"",""AYGA"",-6.081689,145.391881,5282,10,""U"",""Pacific/Port_Moresby"""
"2,""Madang"",""Madang"",""Papua New Guinea"",""MAG"",""AYMD"",-5.207083,145.7887,20,10,""U"",""Pacific/Port_Moresby"""
"3,""Mount Hagen"",""Mount Hagen"",""Papua New Guinea"",""HGU"",""AYMH"",-5.826789,144.295861,5388,10,""U"",""Pacific/Port_Moresby"""
"4,""Nadzab"",""Nadzab"",""Papua New Guinea"",""LAE"",""AYNZ"",-6.569828,146.726242,239,10,""U"",""Pacific/Port_Moresby"""
"5,""Port Moresby Jacksons Intl"",""Port Moresby"",""Papua New Guinea"",""POM"",""AYPY"",-9.443383,147.22005,146,10,""U"",""Pacific/Port_Moresby"""
"6,""Wewak Intl"",""Wewak"",""Papua New Guinea"",""WWK"",""AYWK"",-3.583828,143.669186,19,10,""U"",""Pacific/Port_Moresby"""
"7,""Narsarsuaq"",""Narssarssuaq"",""Greenland"",""UAK"",""BGBW"",61.160517,-45.425978,112,-3,""E"",""America/Godthab"""
"8,""Nuuk"",""Godthaab"",""Greenland"",""GOH"",""BGGH"",64.190922,-51.678064,283,-3,""E"",""America/Godthab"""
"9,""Sondre Stromfjord"",""Sondrestrom"",""Greenland"",""SFJ"",""BGSF"",67.016969,-50.689325,165,-3,""E"",""America/Godthab"""


We can look at the schema of the file using the `describe` function

In [5]:
%%query 
describe("s3://raw-tutorial/inference/airports.csv")

Actually we could just use the `read` function to get the parsed content of the file.

In [83]:
%%query
select * from read("s3://raw-tutorial//inference/airports.csv") limit 10

AirportID,Name,City,Country,IATA_FAA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ
1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10.0,U,Pacific/Port_Moresby
2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10.0,U,Pacific/Port_Moresby
3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10.0,U,Pacific/Port_Moresby
4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10.0,U,Pacific/Port_Moresby
5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10.0,U,Pacific/Port_Moresby
6,Wewak Intl,Wewak,Papua New Guinea,WWK,AYWK,-3.583828,143.669186,19,10.0,U,Pacific/Port_Moresby
7,Narsarsuaq,Narssarssuaq,Greenland,UAK,BGBW,61.160517,-45.425978,112,-3.0,E,America/Godthab
8,Nuuk,Godthaab,Greenland,GOH,BGGH,64.190922,-51.678064,283,-3.0,E,America/Godthab
9,Sondre Stromfjord,Sondrestrom,Greenland,SFJ,BGSF,67.016969,-50.689325,165,-3.0,E,America/Godthab
10,Thule Air Base,Thule,Greenland,THU,BGTL,76.531203,-68.703161,251,-4.0,E,America/Thule


We can also get the schema for every file in this folder with the following query.

In [33]:
%%query
select file, 
    describe(file)
from file in ls("s3://raw-tutorial/inference")


We can see that we have several formats in here:
 * A csv, json and xml file.
 * A text/csv file separated by `|` without header, the system just named each column _1 .. _6.
 * A log file where the system could find extract two fields, `timestamp` and `message`.
 * A parquet file.

## Finding files with errors


Now lets look at another folder.
This is a sample of the [NY Yellow Taxi Trip Data](https://data.cityofnewyork.us/Transportation/2018-Yellow-Taxi-Trip-Data/t29m-gskq#__sid=js0). 

They are all supposed to be CSV files but some of them have errors and some of them have different fields.

So for instance the following query shows which files do not parse as CSV.

In [77]:
%%query

descriptions := select file, 
                    describe(file) as info
                    from file in ls("file:/home/cesar/development/nyc-taxis-sample");


select * from descriptions where info.format != "csv" limit 5

### Similarity in field names over the CSV files
The following query makes a grouping of how many fields are shared between the first file and all the other. <br>
We can see that some share all the fields (7) but a lot of them do not.

In [48]:
%%query

ref_file := "file:/home/cesar/development/nyc-taxis-sample/fhv_tripdata_2018-01.csv";


columns_of(f: string) := 
    select col_name from describe(f).columns;
    
ref_columns := columns_of(ref_file);

similarity(f: string) := select count(*) from columns_of(f) c where c in ref_columns;

select sim, partition
from f in ls("file:/home/cesar/development/nyc-taxis-sample/")
group by similarity(f) sim
order by sim desc


### Reading the data of a file with an error

By reading a file with an error we find out that the second line is empty, right after the header.

In [87]:
%%query 
select * from 
    read("file:/home/cesar/development/nyc-taxis-sample/yellow_tripdata_2014-08.csv")
    limit 10

string
"vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, pickup_longitude, pickup_latitude, rate_code, store_and_fwd_flag, dropoff_longitude, dropoff_latitude, payment_type, fare_amount, surcharge, mta_tax, tip_amount, tolls_amount, total_amount"
""
"CMT,2014-08-16 14:58:49,2014-08-16 15:15:59,1,2.7000000000000002,-73.946537000000006,40.776812999999997,1,N,-73.976192999999995,40.755625000000002,CSH,14,0,0.5,0,0,14.5"
"CMT,2014-08-16 08:10:48,2014-08-16 08:58:16,3,20.399999999999999,-73.776857000000007,40.645099000000002,1,Y,-73.916248999999993,40.837356999999997,CSH,58.5,0,0.5,0,5.3300000000000001,64.329999999999998"
"CMT,2014-08-16 09:44:07,2014-08-16 09:54:37,1,2.1000000000000001,-73.986585000000005,40.725847999999999,1,N,-73.977157000000005,40.751961000000001,CSH,9.5,0,0.5,0,0,10"
"CMT,2014-08-16 10:46:13,2014-08-16 10:51:25,1,1.3,-73.976290000000006,40.765231,1,N,-73.961484999999996,40.777889000000002,CSH,6,0,0.5,0,0,6.5"
"CMT,2014-08-16 09:27:23,2014-08-16 09:39:37,2,1.7,-73.995248000000004,40.754646000000001,1,Y,-73.995902999999998,40.769201000000002,CSH,10.5,0,0.5,0,0,11"
"CMT,2014-08-16 14:14:16,2014-08-16 14:25:33,2,1.7,-73.991534999999999,40.759863000000003,1,N,-74.005722000000006,40.737558,CSH,10,0,0.5,0,0,10.5"
"CMT,2014-08-16 15:55:16,2014-08-16 16:00:10,1,1,-73.972307000000001,40.794075999999997,1,N,-73.963864999999998,40.807858000000003,CSH,6,0,0.5,0,0,6.5"
"CMT,2014-08-16 14:08:29,2014-08-16 14:32:03,1,9.1999999999999993,-73.967337999999998,40.766008999999997,1,N,-73.872972000000004,40.774487000000001,CSH,28.5,0,0.5,0,0,29"


So by passing the `skip := 2` to `read_csv` we now can see all the fields

In [79]:
%%query 
select * from 
    read_csv("file:/home/cesar/development/nyc-taxis-sample/yellow_tripdata_2014-08.csv", skip := 2)
    limit 5

_1,_2,_3,_4,_5,_6,_7,_8,_9,_10,_11,_12,_13,_14,_15,_16,_17,_18
CMT,2014-08-16 14:58:49,2014-08-16 15:15:59,1,2.7,-73.946537,40.776813,1,N,-73.976193,40.755625,CSH,14.0,0.0,0.5,0.0,0.0,14.5
CMT,2014-08-16 08:10:48,2014-08-16 08:58:16,3,20.4,-73.776857,40.645099,1,Y,-73.916249,40.837357,CSH,58.5,0.0,0.5,0.0,5.33,64.33
CMT,2014-08-16 09:44:07,2014-08-16 09:54:37,1,2.1,-73.986585,40.725848,1,N,-73.977157,40.751961,CSH,9.5,0.0,0.5,0.0,0.0,10.0
CMT,2014-08-16 10:46:13,2014-08-16 10:51:25,1,1.3,-73.97629,40.765231,1,N,-73.961485,40.777889,CSH,6.0,0.0,0.5,0.0,0.0,6.5
CMT,2014-08-16 09:27:23,2014-08-16 09:39:37,2,1.7,-73.995248,40.754646,1,Y,-73.995903,40.769201,CSH,10.5,0.0,0.5,0.0,0.0,11.0


By passing the schema of the file to the `read_csv` function we can correct all the field names

In [80]:
%%query

typealias t := record(
    VendorID: string,
    tpep_pickup_datetime: string,
    tpep_dropoff_datetime: string,
    passenger_count: int,
    trip_distance: double,
    pickup_longitude: double,
    pickup_latitude: double,
    RatecodeID: int,
    store_and_fwd_flag: string,
    dropoff_longitude: double,
    dropoff_latitude: double,
    payment_type: string,
    fare_amount: double,
    extra: double,
    mta_tax: double,
    tip_amount: double,
    tolls_amount: double,
    total_amount: double
);
    
select * from 
    read_csv[t]("file:/home/cesar/development/nyc-taxis-sample/yellow_tripdata_2014-08.csv", skip := 2) 
    limit 5

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount
CMT,2014-08-16 14:58:49,2014-08-16 15:15:59,1,2.7,-73.946537,40.776813,1,N,-73.976193,40.755625,CSH,14.0,0.0,0.5,0.0,0.0,14.5
CMT,2014-08-16 08:10:48,2014-08-16 08:58:16,3,20.4,-73.776857,40.645099,1,Y,-73.916249,40.837357,CSH,58.5,0.0,0.5,0.0,5.33,64.33
CMT,2014-08-16 09:44:07,2014-08-16 09:54:37,1,2.1,-73.986585,40.725848,1,N,-73.977157,40.751961,CSH,9.5,0.0,0.5,0.0,0.0,10.0
CMT,2014-08-16 10:46:13,2014-08-16 10:51:25,1,1.3,-73.97629,40.765231,1,N,-73.961485,40.777889,CSH,6.0,0.0,0.5,0.0,0.0,6.5
CMT,2014-08-16 09:27:23,2014-08-16 09:39:37,2,1.7,-73.995248,40.754646,1,Y,-73.995903,40.769201,CSH,10.5,0.0,0.5,0.0,0.0,11.0
